# Ноутбук требует работы ручками :(

* R_MERGED_STAGE1: имеет флаги SRC in {R1_TRAIN, ..., R3_TEST} , FILENAME, SIGN 
* R_MERGED_STAGE2: имеет filepath, NUMBER 


## Задача ноутбука: объеденить 2 пака датасета в 1

In [ ]:
import pandas as pd
import os
import pathlib
import shutil
import numpy as np

RTSD_PUBLIC_ROOT = pathlib.Path('D:\\Downloads\\rtsd-public\\classification')

PATH_TO_RTDS_R1 = RTSD_PUBLIC_ROOT / 'rtsd-r1.tar/rtsd-r1'
PATH_TO_RTDS_R3 = RTSD_PUBLIC_ROOT / 'rtsd-r3.tar/rtsd-r3'

In [ ]:
PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))

DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

RTSD_MERGE_OUT_DIR = DATA_DIR / 'RTSD_CLASSIFIER_DATASET'
RTSD_MERGE_OUT_DIR.mkdir(parents=True, exist_ok=True)

## Сравним number_to_classes в каждой и частей паков датасета

In [ ]:
RTDS_R1_N2C = pd.read_csv(PATH_TO_RTDS_R1 / 'numbers_to_classes.csv')
RTDS_R1_N2C_DICT = RTDS_R1_N2C.set_index('class_number').to_dict()['sign_class']
display(RTDS_R1_N2C.head())

RTDS_R3_N2C = pd.read_csv(PATH_TO_RTDS_R3 / 'numbers_to_classes.csv')
RTDS_R3_N2C_DICT = RTDS_R3_N2C.set_index('class_number').to_dict()['sign_class']
display(RTDS_R3_N2C.head())

In [ ]:
for key in RTDS_R1_N2C_DICT:
    R1_ITEM = RTDS_R1_N2C_DICT[key]
    R3_ITEM = RTDS_R3_N2C_DICT[key]
    if R1_ITEM != R3_ITEM:
        print('1 MISMATCH FOR', key)
        break;
        
for key in RTDS_R3_N2C_DICT:
    R1_ITEM = RTDS_R1_N2C_DICT[key]
    R3_ITEM = RTDS_R3_N2C_DICT[key]
    if R1_ITEM != R3_ITEM:
        print('2 MISMATCH FOR', key)
        break

### Таблица трансляции 'number_to_classes' не совпадают. Будем использовать соответсвующие таблицы для каждого пака, чтобы привести все к единому ДатаФрейму вида 
|INDEX| FILEPATH | SIGN_CLASS | SET | 
| --- | --- | --- | --- |
|1| ../data/FOLDER1/0001.jp | 1_11 | 'train' | 
| --- | --- | --- | --- | 
...
| --- | --- | --- | --- |
|973| ../data/FOLDER2/000714.jp | 1_31 | 'test' | 

In [ ]:
print(RTDS_R1_N2C_DICT)
print(RTDS_R3_N2C_DICT)

## Рассмотрим распределения TEST/TRAIN для обоих паков

### R1

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

R1_TEST = pd.read_csv(PATH_TO_RTDS_R1 / 'gt_test.csv')
# display(R1_TEST)
R1_TRAIN = pd.read_csv(PATH_TO_RTDS_R1 / 'gt_train.csv')
# display(R1_TRAIN)

_, ax = plt.subplots(nrows=2, ncols=1, figsize=(21, 7))
ax[0].tick_params(labelrotation=90)
ax[1].tick_params(labelrotation=90)

g = sns.countplot(ax=ax[0], x='class_number', data=R1_TEST,  order=sorted(R1_TEST['class_number'].value_counts().index.tolist()));
# g.set_yscale("log")
ax[0].set_title('TEST R1 DISTRIBUTION')

g = sns.countplot(ax=ax[1], x='class_number', data=R1_TRAIN,  order=sorted(R1_TRAIN['class_number'].value_counts().index.tolist()));
# g.set_yscale("log")
ax[1].set_title('TRAIN R1 DISTRIBUTION')


plt.tight_layout()

### R3 

In [ ]:
R3_TEST = pd.read_csv(PATH_TO_RTDS_R3 / 'gt_test.csv')
R3_TRAIN = pd.read_csv(PATH_TO_RTDS_R3 / 'gt_train.csv')


_, ax = plt.subplots(nrows=2, ncols=1, figsize=(21, 7))
ax[0].tick_params(labelrotation=90)
ax[1].tick_params(labelrotation=90)

g = sns.countplot(ax=ax[0], x='class_number', data=R3_TEST,  order=sorted(R3_TEST['class_number'].value_counts().index.tolist()));
ax[0].set_title('TEST R3 DISTRIBUTION')

g = sns.countplot(ax=ax[1], x='class_number', data=R3_TRAIN,  order=sorted(R3_TRAIN['class_number'].value_counts().index.tolist()));
ax[1].set_title('TRAIN R3 DISTRIBUTION')

plt.tight_layout()

## В обоих частях датасета, как в тестовой, так и в тренировочной частях есть сильно преобладающий класс. Необходимо избавится от трендов, однако прежде - объеденим R3 и R1 ДатаФрейми в один, избавившись от деления на тестовую и тренировочную выборки. 

Перейдем от номера класса к знаку в соответствии со словарями 

In [ ]:
R1_TRAIN['class_number'] = R1_TRAIN['class_number'].apply(lambda x: RTDS_R1_N2C_DICT[x])
R1_TEST['class_number'] = R1_TEST['class_number'].apply(lambda x: RTDS_R1_N2C_DICT[x])
R3_TRAIN['class_number'] = R3_TRAIN['class_number'].apply(lambda x: RTDS_R3_N2C_DICT[x])
R3_TEST['class_number'] = R3_TEST['class_number'].apply(lambda x: RTDS_R3_N2C_DICT[x])

Сдеюущая операция может занять много времени. Для выполнения, установить MAKE_ONCE_FLAG равным False

In [ ]:
R_MERGED_FILE = DATA_DIR / 'R_MERGED_STAGE1.csv'
R_MERGED = pd.DataFrame(columns=['SRC', 'FILENAME', 'SIGN'])

MAKE_ONCE_FLAG = True 

if os.path.isfile(R_MERGED_FILE):
    R_MERGED = pd.read_csv(R_MERGED_FILE)
else:
    MAKE_ONCE_FLAG = False
    
if not MAKE_ONCE_FLAG:
    
    for row in R1_TRAIN.itertuples():
        R_MERGED.loc[len(R_MERGED)] = ['R1_TRAIN', row.filename, row.class_number]
    
    print('R1_TRAIN COMPLETED. Current R_MERGED size', len(R_MERGED.index), 'R1_TRAIN size', len(R1_TRAIN.index))
    
    for row in R1_TEST.itertuples():
        R_MERGED.loc[len(R_MERGED)] = ['R1_TEST', row.filename, row.class_number]

    print('R1_TEST COMPLETED. Current R_MERGED size', len(R_MERGED.index), 'R1_TRAIN size', len(R1_TEST.index))
    
    for row in R3_TRAIN.itertuples():
        R_MERGED.loc[len(R_MERGED)] = ['R3_TRAIN', row.filename, row.class_number]
    
    print('R3_TRAIN COMPLETED. Current R_MERGED size', len(R_MERGED.index), 'R3_TRAIN size', len(R3_TRAIN.index))
    
    for row in R3_TEST.itertuples():
        R_MERGED.loc[len(R_MERGED)] = ['R3_TEST', row.filename, row.class_number]
    
    print('R3_TEST COMPLETED. Current R_MERGED size', len(R_MERGED.index), 'R3_TEST size', len(R3_TEST.index))
    
    MAKE_ONCE_FLAG = True
    R_MERGED.to_csv(R_MERGED_FILE, index=False)

R_MERGED_SIZE = len(R_MERGED.index)
OTHER_DFs_SIZE = len(R1_TRAIN.index) + len(R1_TEST.index) + len(R3_TRAIN.index) + len(R3_TEST.index)
assert R_MERGED_SIZE==OTHER_DFs_SIZE

In [ ]:
R_MERGED

In [ ]:
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(21, 7))
ax.tick_params(labelrotation=90)

g = sns.countplot(x='SIGN', data=R_MERGED,  order=sorted(R_MERGED['SIGN'].value_counts().index.tolist()));
ax.set_title('R_MERGED')
plt.tight_layout()

Делаем табличку перевода цифорки в знак

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

SignEncoder = le.fit_transform(R_MERGED['SIGN'])

NUMBER_TO_SIGN_DF_DATA = dict(zip(SignEncoder, R_MERGED['SIGN']))
NUMBER_TO_SIGN_DF = pd.DataFrame(data=NUMBER_TO_SIGN_DF_DATA.items(), columns=['NUMBER', 'SIGN'])

NUMBER_TO_SIGN_DF_DIR = DATA_DIR / 'number_to_sign.csv'
NUMBER_TO_SIGN_DF.to_csv(NUMBER_TO_SIGN_DF_DIR, index=False)

SIGN_TO_NUMBER_DF_DATA = dict(zip(R_MERGED['SIGN'], SignEncoder))
SIGN_TO_NUMBER_DF = pd.DataFrame(data=SIGN_TO_NUMBER_DF_DATA.items(), columns=['SIGN', 'NUMBER'])
SIGN_TO_NUMBER_DF_DIR = DATA_DIR / 'sign_to_number.csv'
SIGN_TO_NUMBER_DF.to_csv(SIGN_TO_NUMBER_DF_DIR, index=False)

In [ ]:
R_MERGED['SIGN'] = R_MERGED['SIGN'].apply(lambda x: SIGN_TO_NUMBER_DF_DATA[x])

In [ ]:
R_MERGED

In [ ]:
NUMBER_TO_SIGN_DF_DATA[R_MERGED.loc[4]['SIGN']]

### Имеем 2 таблички: Знак -> Число, Число -> Знак. Будем дальше использовать их как словарь. 
### Удалим ненужные знаки. Список используемых ниже:

In [ ]:
TARGET_SIGNS = [
        '1_1', '1_6', '1_8', '1_22', '1_31', '1_33', '2_1', '2_2', 
        # ~2_3_1
        '2_3', \
        # /~2_3_1
        '2_4', '2_5', '3_1', 
        # ~3_18_1
        '3_18', 
        # /~3_18_1
        '3_20', '3_21', '3_22', '3_23', \
        # all speed limits
        '3_24_n10', '3_24_n20', '3_24_n30', '3_24_n40', '3_24_n50', '3_24_n60', \
        '3_24_n70', '3_24_n80', '3_24_n90', '3_24_n100', '3_24_n110', '3_24_n120', '3_24_n130', \
        # /all speed limits
        '3_25', '3_27', '3_28', '3_31', '4_1_1', '4_3', '5_5', '5_6', '5_16', 
        '5_19_1',  
        #'5_19_2', \ ~ '5_19_1'
        '5_20', '6_3_2', '6_4', '7_3', '7_4'
    ]

In [ ]:
TARGET_NUMBERS = []
nilen = 0
for TARGET_SIGN in TARGET_SIGNS:
    if TARGET_SIGN in SIGN_TO_NUMBER_DF_DATA:
        TARGET_NUMBERS.append(SIGN_TO_NUMBER_DF_DATA[TARGET_SIGN])
    else:
        print(TARGET_SIGN, 'not found in dict')
        nilen += 1

print('not included count:', nilen)
print('\nTarget numbers:', TARGET_NUMBERS)

In [ ]:
FILEPATH_DICT = {
    'R1_TRAIN': 'R_MERGED/rtsd-r1/train/',
    'R1_TEST': 'R_MERGED/rtsd-r1/test/',
    'R3_TRAIN': 'R_MERGED/rtsd-r3/train/',
    'R3_TEST': 'R_MERGED/rtsd-r3/test/',
                }

R_MERGED_FILE = DATA_DIR / 'R_MERGED_STAGE2.csv'

#if os.path.isfile(R_MERGED_FILE):
#    R_MERGED = pd.read_csv(R_MERGED_FILE)

if 'SRC' in R_MERGED.columns:
    R_MERGED.rename(columns={'FILENAME': 'filepath'}, inplace=True)

    for key in FILEPATH_DICT:
        R_MERGED.loc[R_MERGED['SRC']==key, 'filepath'] = R_MERGED[R_MERGED['SRC']==key]['filepath'].apply(lambda x: FILEPATH_DICT[key] + x)

    R_MERGED.drop('SRC', inplace=True, axis=1)
    

    R_MERGED.to_csv(R_MERGED_FILE, index=False)

In [ ]:
R_MERGED

### Осталось разделить датасет на test, train, valid

In [ ]:
_, ax = plt.subplots(nrows=1, ncols=1, figsize=(21, 7))
ax.tick_params(labelrotation=90)

R_MERGED = R_MERGED[R_MERGED['SIGN'].isin(TARGET_NUMBERS)].reset_index(drop=True)

g = sns.countplot(x='SIGN', data=R_MERGED,  order=sorted(R_MERGED['SIGN'].value_counts().index.tolist()))
ax.set_title('R_MERGED')
plt.tight_layout()

Неприятное распределение

In [ ]:
R_MERGED.groupby(['SIGN']).size()

In [ ]:
R_MERGED_GROUPED = R_MERGED.groupby('SIGN', axis=0)
MEAN_BY_GROUPS = int(np.floor(R_MERGED_GROUPED.size().mean()))
R_MERGED['SET'] = np.nan
print(MEAN_BY_GROUPS)

Undersampling: в каждом знаке проверем, привосходит ли число его представителей среднего (MEAN_BY_GROUPS). 
    
    Если превосходит: берем MEAN_BY_GROUPS элементов, 60% закидываем в тренировку, 20% в валидацию, 20% в тест. Все что выше закидываем в тест.
    Если не превосходит: берем все доступные элементов, 60% закидываем в тренировку, 20% в валидацию, 20% в тест.

In [ ]:
import random

SET_COLUMN_INDEX = R_MERGED.columns.get_loc("SET")

for key, items in R_MERGED_GROUPED.groups.items():
        # print(items)
        items = list(items)     # явно приведем к списку для душевного спокойствия
        random.shuffle(items)   # перемешаем

        # print(key)

        if len(items) > MEAN_BY_GROUPS:
            # выбираем рандомные значения из этой группы в колличестве MEAN_BY_GROUPS*0.6 для train
            # MEAN_BY_GROUPS*0.2 для valid, остальное кинем в test
            # print(int(MEAN_BY_GROUPS*0.8))
            
            TEMP_ITEMS_INCLUDED = items[0:MEAN_BY_GROUPS]
            TEMP_ITEMS_EXCLUDED = items[MEAN_BY_GROUPS::]
            
            TRAIN_GROUP, VALID_GROUP, TEST_GPOUP = np.split(
                TEMP_ITEMS_INCLUDED, 
                [int(len(TEMP_ITEMS_INCLUDED)*0.6), 
                 int(len(TEMP_ITEMS_INCLUDED)*0.8)]
            )
            
            TEST_GPOUP = np.append(TEST_GPOUP, TEMP_ITEMS_EXCLUDED)
            #if key == '3_24_n40':
            #    print('TRAIN', sorted(TRAIN_GROUP), '\nVALID', sorted(VALID_GROUP), '\nTEST', sorted(TEST_GPOUP))
            #    # print('t', len(TRAIN_GROUP), 'v', len(VALID_GROUP), 't', len(TEST_GPOUP))
        else:
            TRAIN_GROUP, VALID_GROUP, TEST_GPOUP = np.split(items, [int(len(items)*0.6), int(len(items)*0.8)])
            # print('t', len(TRAIN_GROUP), 'v', len(VALID_GROUP), 't', len(TEST_GPOUP), '\n')
            # print('t', TRAIN_GROUP, 'v', VALID_GROUP, 't', TEST_GPOUP, '\n')    


        R_MERGED.iloc[TRAIN_GROUP, SET_COLUMN_INDEX] = 'train'
        R_MERGED.iloc[VALID_GROUP, SET_COLUMN_INDEX] = 'valid'
        R_MERGED.iloc[TEST_GPOUP, SET_COLUMN_INDEX] = 'test'

R_MERGED

In [ ]:
_, ax = plt.subplots(nrows=3, ncols=1, figsize=(21, 14))

LABELS = ['train', 'valid', 'test']
for i in range(len(LABELS)):
    g = sns.countplot(x='SIGN', 
                      data=R_MERGED[R_MERGED['SET']==LABELS[i]],  
                      ax=ax[i], 
                      order=sorted(R_MERGED['SIGN'].value_counts().index.tolist())
                     )
    ax[i].tick_params(labelrotation=90)
    ax[i].set_title(LABELS[i])
    plt.tight_layout()

OVERSAMPLING: каждый класс, который не дотягивает до MEAN_BY_GROUPS в обучающей выборке вставляем в обучающую выборку снова

In [ ]:
R_MERGED_TRAIN = R_MERGED[R_MERGED['SET']=='train']
R_MERGED_TRAIN_GROUPED = R_MERGED_TRAIN.groupby('SIGN', axis=0)

for key, items in R_MERGED_TRAIN_GROUPED.groups.items():

    items = list(items)     # явно приведем к списку для душевного спокойствия
    random.shuffle(items)   # перемешаем

    if len(items) < MEAN_BY_GROUPS:
        ROWS_TO_FILL_COUNT = int(MEAN_BY_GROUPS*0.6) - len(items)
        ROWS_TO_APPEND = R_MERGED.iloc[items].sample(ROWS_TO_FILL_COUNT, replace=True)
        # print(ROWS_TO_APPEND)
        #print(len(gt.index))
        R_MERGED = R_MERGED.append(ROWS_TO_APPEND, ignore_index=True)
        #print(len(gt.index))

In [ ]:
R_MERGED

In [ ]:
_, ax = plt.subplots(nrows=3, ncols=1, figsize=(21, 14))

LABELS = ['train', 'valid', 'test']

for i in range(len(LABELS)):
    g = sns.countplot(x='SIGN', 
                      data=R_MERGED[R_MERGED['SET']==LABELS[i]],  
                      ax=ax[i], 
                      order=sorted(R_MERGED['SIGN'].value_counts().index.tolist())
                     )
    ax[i].tick_params(labelrotation=90)
    ax[i].set_title(LABELS[i])
    plt.tight_layout()

In [ ]:
R_MERGED_FILE = DATA_DIR / 'RTDS_DATASET.csv'
R_MERGED.to_csv(R_MERGED_FILE, index=False)

In [ ]:
assert False, 'BREAKPOINT'

### Работа ручками: объеденить папки исходные rtsd-r1 и rtsd-r3 в одну папку R_MERGED. Около папки должен дежать RTDS_DATASET.csv

In [ ]:
R_MERGED_FILE = DATA_DIR / 'RTDS_DATASET.csv'
R_MERGED = pd.read_csv(R_MERGED_FILE)

_, ax = plt.subplots(nrows=3, ncols=1, figsize=(21, 14))

LABELS = ['train', 'valid', 'test']

for i in range(len(LABELS)):
    g = sns.countplot(x='SIGN', 
                      data=R_MERGED[R_MERGED['SET']==LABELS[i]],  
                      ax=ax[i], 
                      order=sorted(R_MERGED['SIGN'].value_counts().index.tolist())
                     )
    ax[i].tick_params(labelrotation=90)
    ax[i].set_title(LABELS[i])
    plt.tight_layout()